# Phase 2: Risk Management & Position Sizing

The most important phase. Successful traders prioritize risk controls above all else.

This notebook covers:
- **Stop-loss and take-profit** mechanics
- **Position sizing** — Kelly criterion, fixed fractional, volatility-based
- **Leverage controls** — conservative limits, auto de-risking
- **Drawdown protection** — circuit breakers, equity curve monitoring
- **Risk metrics** — Sharpe ratio, Sortino, max drawdown, win rate

---

## Prerequisites

```bash
pip install pandas numpy matplotlib scipy
```

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from dataclasses import dataclass
from typing import Optional

plt.style.use('seaborn-v0_8-darkgrid')
np.random.seed(42)

# Generate sample price data
def generate_prices(n=500, mu=0.08, sigma=0.25, start=100.0):
    dt = 1/252
    returns = np.random.normal(mu * dt, sigma * np.sqrt(dt), n)
    prices = start * np.exp(np.cumsum(returns))
    dates = pd.date_range('2023-01-01', periods=n, freq='B')
    return pd.Series(prices, index=dates, name='close')

prices = generate_prices()
returns = prices.pct_change().dropna()
print(f"Price range: ${prices.min():.2f} — ${prices.max():.2f}")
print(f"Daily return stats: mean={returns.mean():.4f}, std={returns.std():.4f}")

---
## 2.1 Stop-Loss & Take-Profit Mechanics

Stop-losses are the single most important risk tool. They limit how much you can lose on any single trade.

**Types of stops:**
- **Fixed stop**: Set at a fixed distance from entry (e.g., 2% below)
- **ATR-based stop**: Set at a multiple of Average True Range (adapts to volatility)
- **Trailing stop**: Follows price up, locks in profits
- **Time stop**: Exit after N candles regardless of price

In [ ]:
def simulate_trade_with_stops(
    prices: pd.Series,
    entry_idx: int,
    side: str = 'long',
    stop_loss_pct: float = 0.02,     # 2% stop loss
    take_profit_pct: float = 0.06,   # 6% take profit (3:1 reward:risk)
    trailing_stop_pct: float = None,  # optional trailing stop
    max_hold_candles: int = 60        # time stop
) -> dict:
    """Simulate a single trade with stop-loss, take-profit, and trailing stop."""
    entry_price = prices.iloc[entry_idx]
    direction = 1 if side == 'long' else -1

    stop_price = entry_price * (1 - direction * stop_loss_pct)
    tp_price = entry_price * (1 + direction * take_profit_pct)
    highest = entry_price  # for trailing stop

    price_path = []
    stop_path = []

    for i in range(entry_idx + 1, min(entry_idx + max_hold_candles, len(prices))):
        current = prices.iloc[i]
        price_path.append(current)

        # Update trailing stop
        if trailing_stop_pct and side == 'long' and current > highest:
            highest = current
            stop_price = max(stop_price, highest * (1 - trailing_stop_pct))
        elif trailing_stop_pct and side == 'short' and current < highest:
            highest = current
            stop_price = min(stop_price, highest * (1 + trailing_stop_pct))

        stop_path.append(stop_price)

        # Check stop loss
        if (side == 'long' and current <= stop_price) or \
           (side == 'short' and current >= stop_price):
            pnl_pct = direction * (current - entry_price) / entry_price
            return {'exit': 'stop_loss', 'entry': entry_price, 'exit_price': current,
                    'pnl_pct': pnl_pct, 'candles_held': i - entry_idx,
                    'price_path': price_path, 'stop_path': stop_path}

        # Check take profit
        if (side == 'long' and current >= tp_price) or \
           (side == 'short' and current <= tp_price):
            pnl_pct = direction * (current - entry_price) / entry_price
            return {'exit': 'take_profit', 'entry': entry_price, 'exit_price': current,
                    'pnl_pct': pnl_pct, 'candles_held': i - entry_idx,
                    'price_path': price_path, 'stop_path': stop_path}

    # Time stop
    exit_price = prices.iloc[min(entry_idx + max_hold_candles, len(prices) - 1)]
    pnl_pct = direction * (exit_price - entry_price) / entry_price
    return {'exit': 'time_stop', 'entry': entry_price, 'exit_price': exit_price,
            'pnl_pct': pnl_pct, 'candles_held': max_hold_candles,
            'price_path': price_path, 'stop_path': stop_path}


# Run many trades with fixed stops
trade_results = []
for start in range(0, len(prices) - 70, 5):
    result = simulate_trade_with_stops(
        prices, start,
        stop_loss_pct=0.02,
        take_profit_pct=0.06,
        max_hold_candles=60
    )
    trade_results.append(result)

results_df = pd.DataFrame(trade_results)
print(f"Total trades: {len(results_df)}")
print(f"\nExit reasons:")
print(results_df['exit'].value_counts())
print(f"\nAvg P&L: {results_df['pnl_pct'].mean():.2%}")
print(f"Win rate: {(results_df['pnl_pct'] > 0).mean():.1%}")
print(f"Avg winner: {results_df[results_df['pnl_pct'] > 0]['pnl_pct'].mean():.2%}")
print(f"Avg loser: {results_df[results_df['pnl_pct'] <= 0]['pnl_pct'].mean():.2%}")

In [ ]:
# Visualize a single trade with trailing stop
result = simulate_trade_with_stops(
    prices, entry_idx=50,
    stop_loss_pct=0.03,
    take_profit_pct=0.10,
    trailing_stop_pct=0.025,
    max_hold_candles=80
)

fig, ax = plt.subplots(figsize=(14, 6))
x = range(len(result['price_path']))
ax.plot(x, result['price_path'], color='steelblue', label='Price')
ax.plot(x, result['stop_path'], color='red', linestyle='--', label='Trailing Stop')
ax.axhline(y=result['entry'], color='green', linestyle=':', alpha=0.7, label=f"Entry ${result['entry']:.2f}")
ax.axhline(y=result['entry'] * 1.10, color='blue', linestyle=':', alpha=0.7, label='Take Profit (10%)')

exit_color = 'green' if result['pnl_pct'] > 0 else 'red'
ax.scatter(len(result['price_path'])-1, result['exit_price'], color=exit_color, s=100, zorder=5,
           label=f"Exit: {result['exit']} ({result['pnl_pct']:.2%})")

ax.set_title('Trade with Trailing Stop', fontsize=14)
ax.set_xlabel('Candles held')
ax.set_ylabel('Price ($)')
ax.legend()
plt.tight_layout()
plt.show()

### Exercise 2.1

1. Compare reward:risk ratios of 1:1, 2:1, 3:1, and 5:1 — which gives the highest total P&L over all trades?
2. Add trailing stops to the batch simulation. How does a 2.5% trailing stop affect win rate vs avg winner size?
3. What happens if you set stop_loss_pct=0.005 (0.5%) with high volatility data? Why?

In [ ]:
# YOUR CODE HERE


---
## 2.2 Position Sizing Methods

Position sizing determines HOW MUCH to risk on each trade. It's arguably more important than the strategy itself.

### Methods:
1. **Fixed Fractional**: Risk a fixed % of equity per trade
2. **Kelly Criterion**: Optimal sizing based on win rate and payoff ratio
3. **Volatility-Based (ATR)**: Size inversely proportional to volatility
4. **Fixed Dollar**: Risk a fixed dollar amount per trade

In [ ]:
class PositionSizer:
    """Calculate position sizes using different methods."""

    @staticmethod
    def fixed_fractional(equity: float, risk_pct: float, stop_distance_pct: float,
                         price: float, leverage: float = 1.0) -> dict:
        """Risk a fixed percentage of equity per trade."""
        risk_amount = equity * risk_pct
        quantity = risk_amount / (price * stop_distance_pct)
        position_value = quantity * price
        margin_needed = position_value / leverage
        return {
            'method': 'fixed_fractional',
            'quantity': quantity,
            'position_value': position_value,
            'risk_amount': risk_amount,
            'margin_needed': margin_needed,
            'effective_leverage': position_value / equity
        }

    @staticmethod
    def kelly_criterion(win_rate: float, avg_win: float, avg_loss: float) -> float:
        """Calculate Kelly fraction: f* = W/A - (1-W)/B
        where W=win rate, A=avg loss, B=avg win."""
        if avg_loss == 0:
            return 0.0
        b = avg_win / abs(avg_loss)  # payoff ratio
        kelly = win_rate - (1 - win_rate) / b
        return max(0, kelly)

    @staticmethod
    def volatility_based(equity: float, risk_pct: float, atr: float,
                          atr_multiplier: float, price: float,
                          leverage: float = 1.0) -> dict:
        """Size position inversely proportional to volatility (ATR)."""
        risk_amount = equity * risk_pct
        stop_distance = atr * atr_multiplier
        quantity = risk_amount / stop_distance
        position_value = quantity * price
        margin_needed = position_value / leverage
        return {
            'method': 'volatility_based',
            'quantity': quantity,
            'position_value': position_value,
            'risk_amount': risk_amount,
            'stop_distance': stop_distance,
            'margin_needed': margin_needed,
            'effective_leverage': position_value / equity
        }


sizer = PositionSizer()

# Compare sizing methods
equity = 50_000
price = prices.iloc[-1]
atr = returns.std() * price * np.sqrt(14)  # rough 14-day ATR proxy

print(f"Account equity: ${equity:,.2f}")
print(f"Current price:  ${price:.2f}")
print(f"Est. 14d ATR:   ${atr:.2f}")
print()

# Fixed fractional: risk 1% with 2% stop
ff = sizer.fixed_fractional(equity, risk_pct=0.01, stop_distance_pct=0.02, price=price, leverage=5)
print("=== Fixed Fractional (1% risk, 2% stop, 5x leverage) ===")
for k, v in ff.items():
    if isinstance(v, float):
        print(f"  {k}: ${v:,.2f}" if 'value' in k or 'amount' in k or 'margin' in k else f"  {k}: {v:.4f}")
print()

# Kelly criterion
kelly_f = sizer.kelly_criterion(win_rate=0.45, avg_win=0.06, avg_loss=0.02)
print(f"=== Kelly Criterion ===")
print(f"  Win rate: 45%, Avg win: 6%, Avg loss: 2%")
print(f"  Full Kelly: {kelly_f:.1%}")
print(f"  Half Kelly (recommended): {kelly_f/2:.1%}")
print(f"  Quarter Kelly (conservative): {kelly_f/4:.1%}")
print()

# Volatility-based
vb = sizer.volatility_based(equity, risk_pct=0.01, atr=atr, atr_multiplier=2, price=price, leverage=5)
print("=== Volatility-Based (1% risk, 2x ATR stop, 5x leverage) ===")
for k, v in vb.items():
    if isinstance(v, float):
        print(f"  {k}: ${v:,.2f}" if 'value' in k or 'amount' in k or 'margin' in k or 'distance' in k else f"  {k}: {v:.4f}")

In [ ]:
# Kelly Criterion sensitivity analysis
win_rates = np.arange(0.30, 0.71, 0.01)
payoff_ratios = [1.0, 1.5, 2.0, 3.0, 5.0]

fig, ax = plt.subplots(figsize=(12, 6))
for pr in payoff_ratios:
    kelly_values = [max(0, wr - (1 - wr) / pr) for wr in win_rates]
    ax.plot(win_rates * 100, [k * 100 for k in kelly_values], label=f'Payoff ratio {pr:.1f}:1')

ax.set_xlabel('Win Rate (%)')
ax.set_ylabel('Kelly Fraction (%)')
ax.set_title('Kelly Criterion: Optimal Position Size', fontsize=14)
ax.legend()
ax.axhline(y=0, color='red', linewidth=0.5)
ax.set_xlim(30, 70)
ax.set_ylim(-5, 60)
plt.tight_layout()
plt.show()
print("Note: Full Kelly is aggressive. Most practitioners use Half Kelly or less.")

### Exercise 2.2

1. With $100,000 equity, 2% risk per trade, and a 3% stop-loss, how many shares of a $50 stock can you buy? Verify with the `fixed_fractional` method.
2. Your strategy has a 40% win rate with average win of $500 and average loss of $200. What's the Kelly fraction? Should you trade this strategy?
3. Compare position sizes for BTC ($40,000, ATR=$2,000) vs a stock ($150, ATR=$3) using the volatility method with the same risk budget.

In [ ]:
# YOUR CODE HERE


---
## 2.3 Drawdown Analysis & Circuit Breakers

Drawdown is the peak-to-trough decline in equity. It measures how much you've lost from the highest point. Managing drawdowns is critical — a 50% drawdown requires a 100% gain to recover.

In [ ]:
def calculate_drawdown(equity_curve: pd.Series) -> pd.DataFrame:
    """Calculate drawdown series from an equity curve."""
    peak = equity_curve.expanding().max()
    drawdown = (equity_curve - peak) / peak
    return pd.DataFrame({
        'equity': equity_curve,
        'peak': peak,
        'drawdown': drawdown,
        'drawdown_dollars': equity_curve - peak
    })


def recovery_required(drawdown_pct: float) -> float:
    """Calculate the gain required to recover from a drawdown."""
    return 1 / (1 + drawdown_pct) - 1  # drawdown_pct is negative


# Simulate an equity curve from our trades
equity = 100_000
leverage = 5
risk_per_trade = 0.01
equity_curve = [equity]

for _, r in results_df.iterrows():
    # Size based on fixed fractional, amplified by leverage
    trade_return = r['pnl_pct'] * leverage * risk_per_trade / 0.02  # scale to risk budget
    equity *= (1 + trade_return)
    equity_curve.append(equity)

equity_series = pd.Series(equity_curve, name='equity')
dd_df = calculate_drawdown(equity_series)

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 8), sharex=True)

ax1.plot(dd_df['equity'], color='steelblue', label='Equity')
ax1.plot(dd_df['peak'], color='gray', linestyle='--', alpha=0.5, label='Peak')
ax1.set_ylabel('Equity ($)')
ax1.set_title('Equity Curve & Drawdown Analysis', fontsize=14)
ax1.legend()

ax2.fill_between(dd_df.index, dd_df['drawdown'] * 100, 0, color='red', alpha=0.3)
ax2.plot(dd_df['drawdown'] * 100, color='red', linewidth=0.5)
ax2.axhline(y=-10, color='orange', linestyle='--', alpha=0.7, label='10% circuit breaker')
ax2.axhline(y=-20, color='red', linestyle='--', alpha=0.7, label='20% circuit breaker')
ax2.set_ylabel('Drawdown (%)')
ax2.set_xlabel('Trade #')
ax2.legend()

plt.tight_layout()
plt.show()

print(f"Final equity: ${equity_curve[-1]:,.2f}")
print(f"Max drawdown: {dd_df['drawdown'].min():.2%}")
print(f"Recovery needed from max DD: {recovery_required(dd_df['drawdown'].min()):.2%}")

In [ ]:
# The drawdown recovery table — why risk management matters
drawdowns = [5, 10, 15, 20, 25, 30, 40, 50, 60, 70, 80, 90]
print(f"{'Drawdown':>10} {'Recovery Needed':>18} {'At 10%/yr':>12} {'At 20%/yr':>12}")
print('-' * 55)
for dd in drawdowns:
    recovery = 1 / (1 - dd/100) - 1
    years_10 = np.log(1 + recovery) / np.log(1.10) if recovery > 0 else 0
    years_20 = np.log(1 + recovery) / np.log(1.20) if recovery > 0 else 0
    print(f"{dd:>9}% {recovery:>17.1%} {years_10:>10.1f} yrs {years_20:>10.1f} yrs")

print("\nKey insight: keeping drawdowns under 20% is critical for long-term survival.")

In [ ]:
class CircuitBreaker:
    """Monitors equity and halts trading when drawdown limits are hit."""

    def __init__(self,
                 max_daily_loss_pct: float = 0.03,
                 max_drawdown_pct: float = 0.15,
                 max_consecutive_losses: int = 5,
                 cooldown_candles: int = 20):
        self.max_daily_loss_pct = max_daily_loss_pct
        self.max_drawdown_pct = max_drawdown_pct
        self.max_consecutive_losses = max_consecutive_losses
        self.cooldown_candles = cooldown_candles

        self.peak_equity = 0
        self.daily_start_equity = 0
        self.consecutive_losses = 0
        self.cooldown_remaining = 0
        self.triggered_reasons = []

    def reset_daily(self, equity: float):
        self.daily_start_equity = equity

    def check(self, equity: float) -> dict:
        self.peak_equity = max(self.peak_equity, equity)
        if self.daily_start_equity == 0:
            self.daily_start_equity = equity

        if self.cooldown_remaining > 0:
            self.cooldown_remaining -= 1
            return {'allow_trading': False, 'reason': f'Cooldown ({self.cooldown_remaining} remaining)'}

        drawdown = (equity - self.peak_equity) / self.peak_equity
        daily_loss = (equity - self.daily_start_equity) / self.daily_start_equity

        if abs(drawdown) >= self.max_drawdown_pct:
            self.cooldown_remaining = self.cooldown_candles
            reason = f'Max drawdown hit: {drawdown:.2%}'
            self.triggered_reasons.append(reason)
            return {'allow_trading': False, 'reason': reason}

        if abs(daily_loss) >= self.max_daily_loss_pct:
            self.cooldown_remaining = self.cooldown_candles
            reason = f'Daily loss limit hit: {daily_loss:.2%}'
            self.triggered_reasons.append(reason)
            return {'allow_trading': False, 'reason': reason}

        if self.consecutive_losses >= self.max_consecutive_losses:
            self.cooldown_remaining = self.cooldown_candles
            reason = f'Consecutive losses: {self.consecutive_losses}'
            self.triggered_reasons.append(reason)
            self.consecutive_losses = 0
            return {'allow_trading': False, 'reason': reason}

        return {'allow_trading': True, 'drawdown': drawdown, 'daily_loss': daily_loss}

    def record_trade(self, pnl: float):
        if pnl < 0:
            self.consecutive_losses += 1
        else:
            self.consecutive_losses = 0


# Demo the circuit breaker
cb = CircuitBreaker(max_daily_loss_pct=0.03, max_drawdown_pct=0.10, max_consecutive_losses=4)
equity = 100_000
cb.peak_equity = equity
cb.daily_start_equity = equity

sample_trades = [-200, -150, -300, -180, 500, -400, -350, -600, -500, -100, 800, 200]

print(f"{'Trade':>8} {'Equity':>12} {'Status':>10} {'Details'}")
print('-' * 60)
for i, trade_pnl in enumerate(sample_trades):
    status = cb.check(equity)
    if status['allow_trading']:
        equity += trade_pnl
        cb.record_trade(trade_pnl)
        print(f"${trade_pnl:>+7,.0f} ${equity:>11,.2f} {'TRADED':>10}  dd={status['drawdown']:.2%}")
    else:
        print(f"${trade_pnl:>+7,.0f} ${equity:>11,.2f} {'BLOCKED':>10}  {status['reason']}")

### Exercise 2.3

1. What is the maximum drawdown if you risk 2% per trade and lose 10 trades in a row? Calculate: `1 - (0.98)^10`
2. Modify the circuit breaker to also track a "weekly loss limit" of 5%.
3. Run the equity curve simulation with and without circuit breakers. Does the circuit breaker improve the final equity?

In [ ]:
# YOUR CODE HERE


---
## 2.4 Risk Metrics Dashboard

Key metrics every trader should monitor continuously.

In [ ]:
def calculate_risk_metrics(trade_returns: np.ndarray, risk_free_rate: float = 0.04) -> dict:
    """Calculate comprehensive risk metrics from a series of trade returns."""
    n_trades = len(trade_returns)
    winners = trade_returns[trade_returns > 0]
    losers = trade_returns[trade_returns <= 0]

    # Annualize (assume ~252 trades per year for daily)
    ann_factor = np.sqrt(252)
    ann_return = np.mean(trade_returns) * 252
    ann_vol = np.std(trade_returns) * ann_factor

    # Sharpe ratio
    sharpe = (ann_return - risk_free_rate) / ann_vol if ann_vol > 0 else 0

    # Sortino ratio (only downside deviation)
    downside = trade_returns[trade_returns < 0]
    downside_std = np.std(downside) * ann_factor if len(downside) > 0 else 0.0001
    sortino = (ann_return - risk_free_rate) / downside_std

    # Profit factor
    gross_profit = winners.sum() if len(winners) > 0 else 0
    gross_loss = abs(losers.sum()) if len(losers) > 0 else 0.0001
    profit_factor = gross_profit / gross_loss

    # Expectancy
    win_rate = len(winners) / n_trades if n_trades > 0 else 0
    avg_win = winners.mean() if len(winners) > 0 else 0
    avg_loss = abs(losers.mean()) if len(losers) > 0 else 0
    expectancy = (win_rate * avg_win) - ((1 - win_rate) * avg_loss)

    # Max consecutive losses
    max_consec_loss = 0
    current_streak = 0
    for r in trade_returns:
        if r <= 0:
            current_streak += 1
            max_consec_loss = max(max_consec_loss, current_streak)
        else:
            current_streak = 0

    return {
        'Total Trades': n_trades,
        'Win Rate': f"{win_rate:.1%}",
        'Avg Win': f"{avg_win:.2%}",
        'Avg Loss': f"-{avg_loss:.2%}",
        'Payoff Ratio': f"{avg_win/avg_loss:.2f}:1" if avg_loss > 0 else 'N/A',
        'Profit Factor': f"{profit_factor:.2f}",
        'Expectancy': f"{expectancy:.4f}",
        'Ann. Return': f"{ann_return:.1%}",
        'Ann. Volatility': f"{ann_vol:.1%}",
        'Sharpe Ratio': f"{sharpe:.2f}",
        'Sortino Ratio': f"{sortino:.2f}",
        'Max Consec. Losses': max_consec_loss,
        'Kelly Fraction': f"{PositionSizer.kelly_criterion(win_rate, avg_win, avg_loss):.1%}"
    }


# Calculate metrics from our simulated trades
trade_returns = results_df['pnl_pct'].values
metrics = calculate_risk_metrics(trade_returns)

print("=" * 40)
print("   RISK METRICS DASHBOARD")
print("=" * 40)
for k, v in metrics.items():
    print(f"  {k:<22} {v:>15}")
print("=" * 40)

In [ ]:
# Distribution of trade returns
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Return distribution
axes[0].hist(trade_returns * 100, bins=30, color='steelblue', alpha=0.7, edgecolor='white')
axes[0].axvline(x=0, color='red', linewidth=1)
axes[0].axvline(x=np.mean(trade_returns) * 100, color='green', linestyle='--', label=f'Mean: {np.mean(trade_returns):.2%}')
axes[0].set_xlabel('Return (%)')
axes[0].set_ylabel('Count')
axes[0].set_title('Trade Return Distribution')
axes[0].legend()

# Cumulative returns
cum_returns = np.cumproduct(1 + trade_returns)
axes[1].plot(cum_returns, color='steelblue')
axes[1].axhline(y=1.0, color='gray', linestyle='--', alpha=0.5)
axes[1].set_xlabel('Trade #')
axes[1].set_ylabel('Cumulative Return')
axes[1].set_title('Cumulative Returns')

# Win/loss streaks
streak_lengths = []
streak_types = []
current = 0
current_type = None
for r in trade_returns:
    t = 'win' if r > 0 else 'loss'
    if t == current_type:
        current += 1
    else:
        if current_type is not None:
            streak_lengths.append(current)
            streak_types.append(current_type)
        current = 1
        current_type = t
streak_lengths.append(current)
streak_types.append(current_type)

colors = ['green' if t == 'win' else 'red' for t in streak_types]
axes[2].bar(range(len(streak_lengths)), streak_lengths, color=colors, alpha=0.7)
axes[2].set_xlabel('Streak #')
axes[2].set_ylabel('Length')
axes[2].set_title('Win/Loss Streaks')

plt.tight_layout()
plt.show()

### Exercise 2.4

1. A strategy has a Sharpe ratio of 0.8. Is it good? What about 1.5? 2.5? Research typical Sharpe ratios.
2. Why is Sortino ratio often preferred over Sharpe? Hint: think about upside vs downside volatility.
3. Your strategy has a win rate of 35% but a payoff ratio of 4:1. Calculate the expectancy. Is this viable?

In [ ]:
# YOUR CODE HERE


---
## 2.5 Leverage vs Risk: The Efficient Frontier

How does changing leverage affect the risk-adjusted return? There's an optimal leverage level beyond which adding more leverage actually reduces expected long-term wealth (due to volatility drag).

In [ ]:
# Simulate the effect of leverage on long-term growth
leverage_range = np.arange(1, 30, 0.5)
n_simulations = 200
n_years = 10
annual_return = 0.10  # 10% unleveraged
annual_vol = 0.20     # 20% unleveraged

median_final_wealth = []
mean_final_wealth = []
prob_ruin = []

for lev in leverage_range:
    lev_return = annual_return * lev
    lev_vol = annual_vol * lev

    # Geometric (log) growth rate = mu - sigma^2/2
    log_growth = lev_return - (lev_vol ** 2) / 2

    # Simulate many paths
    final_values = []
    ruin_count = 0
    for _ in range(n_simulations):
        daily_returns = np.random.normal(
            lev_return / 252,
            lev_vol / np.sqrt(252),
            252 * n_years
        )
        equity = 1.0
        ruined = False
        for r in daily_returns:
            equity *= (1 + r)
            if equity <= 0.01:  # 99% loss = ruin
                ruined = True
                break
        final_values.append(equity if not ruined else 0.01)
        if ruined:
            ruin_count += 1

    median_final_wealth.append(np.median(final_values))
    mean_final_wealth.append(np.mean(final_values))
    prob_ruin.append(ruin_count / n_simulations)

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 9))

ax1.plot(leverage_range, median_final_wealth, color='steelblue', linewidth=2, label='Median wealth')
ax1.plot(leverage_range, mean_final_wealth, color='orange', linewidth=1, alpha=0.7, label='Mean wealth')
opt_lev_idx = np.argmax(median_final_wealth)
ax1.axvline(x=leverage_range[opt_lev_idx], color='green', linestyle='--',
            label=f'Optimal leverage: {leverage_range[opt_lev_idx]:.1f}x')
ax1.set_ylabel(f'Final Wealth (starting from $1, after {n_years} years)')
ax1.set_title(f'Leverage vs Long-Term Wealth ({annual_return:.0%} return, {annual_vol:.0%} vol)', fontsize=14)
ax1.legend()
ax1.set_yscale('log')

ax2.plot(leverage_range, [p * 100 for p in prob_ruin], color='red', linewidth=2)
ax2.set_xlabel('Leverage')
ax2.set_ylabel('Probability of Ruin (%)')
ax2.set_title('Leverage vs Probability of Ruin (99%+ loss)')

plt.tight_layout()
plt.show()

print(f"Optimal leverage (max median wealth): {leverage_range[opt_lev_idx]:.1f}x")
print(f"Median wealth at optimal: ${median_final_wealth[opt_lev_idx]:.2f}")
print(f"Ruin probability at optimal: {prob_ruin[opt_lev_idx]:.1%}")
print(f"\nTheoretical optimal (Kelly): {annual_return / annual_vol**2:.1f}x")

### Exercise 2.5

1. Change `annual_vol` to 0.50 (crypto-like). How does the optimal leverage change?
2. The theoretical Kelly optimal leverage is `mu / sigma^2`. Verify this matches the simulation.
3. Why does the mean wealth keep rising with leverage while median wealth peaks and falls? What does this tell you about the distribution of outcomes?

In [ ]:
# YOUR CODE HERE


---
## 2.6 Comprehension Check

1. You have $25,000 and want to risk 1% per trade. Your stop loss is 3% from entry. What position size should you take?
2. Your strategy wins 50% of the time with an average win of 3% and average loss of 2%. What is the Kelly fraction? What position size would Half Kelly suggest?
3. After a 30% drawdown, what percentage gain is needed to recover? How long at 15% annual returns?
4. Why do successful traders typically use 1/4 to 1/2 Kelly instead of full Kelly?
5. With 20% annual volatility, what leverage maximizes long-term geometric growth if the expected return is 12%?

In [ ]:
# YOUR ANSWERS / CALCULATIONS HERE
